In [1]:
import sys

sys.path.append('..')

In [2]:
from VisionSystem import *
from VisionSystem.DetectionModel import *

def load_or_create_new_threshblob(path):
    try:
        model = ThreshBlob.load(path)
        print("Loaded " + path)
    except Exception:
        model = ThreshBlob()
    finally:
        return model

model_names = ["ball", "obstacle"] # "blue_goal", "yellow_goal", 
detection_models = {
    model_name: load_or_create_new_threshblob("../models/" + model_name + ".threshblob.pkl") \
                    for model_name in model_names
}

objects_to_track = {
    name: VisualObject(real_size=(0.043, 0.043, 0.043), detection_model=model, result_limit=1)
            for name, model in detection_models.items()
}

stream = VideoStream(video_path='../data/recordings/1.h264', crop=((0.13, 0), (.9, 1)))

Loaded ../models/ball.threshblob.pkl
Loaded ../models/obstacle.threshblob.pkl


In [3]:
stream.resolution

(62, 60)

In [4]:
# import cv2
# bgr = cv2.imread('../data/calibration/2.png')
# crop = (0.13, 0), (.9, 1)
# height, width, _ = bgr.shape
# bgr = bgr[:, int(0.13 * width):int(.9 * width)]
vision_system = VisionSystem(objects_to_track, resolution=stream.resolution)

In [10]:
import cv2
cap = cv2.VideoCapture('../data/recordings/1.mp4')
cap.get(cv2.CAP_PROP_FRAME_COUNT)

848.0

In [8]:
from DisplayPane import DisplayPane
from DisplayPane.Interactor.VisionSystemTuner import VisionSystemTuner

display = DisplayPane(
    video_stream=stream,
    vision_system=vision_system,
    interactors=[VisionSystemTuner(vision_system)]
)
display

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# RUN THIS CELL TO SAVE THE MODELs tinkered with
for model_name in model_names:
    detection_models[model_name].save("../models/" + model_name + ".threshblob.pkl")